In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# hide
from nbdev import *

In [ ]:
# hide
from ipyannotator.annotator import Annotator
from ipyannotator.base import Settings
from ipyannotator.mltypes import InputImage, OutputVideoBbox, NoOutput
from ipyannotator.datasets.factory import DS, get_settings
from ipyannotator.helpers import Tutorial

# Tutorial: Video Annotator

The current notebook will demonstrate how to use Ipyannotator to explore, create and improve video annotation.

It's used an artifical video dataset that follows [MOT data format](https://github.com/JonathonLuiten/TrackEval/blob/master/docs/MOTChallenge-Official/Readme.md#data-format).

## Select dataset

In [ ]:
dataset = DS.ARTIFICIAL_VIDEO

## Setup annotator

This section will instantiate the BBoxVideoAnnotator using Ipyannotator's input/output factory. This annotator uses a image as input and output's a bbox video UI that handles labelling (in this case with the classes `circle` and `rectangle`).

In [ ]:
settings_ = get_settings(dataset)
settings_.project_file, settings_.image_dir

In [ ]:
input_ = InputImage(image_dir=settings_.image_dir,
                    image_width=settings_.im_width,
                    image_height=settings_.im_height)

output_ = OutputVideoBbox(classes=['Circle', 'Rectangle'])

input_.dir

In [ ]:
anni = Annotator(input_, NoOutput(), settings_)

## Explore

Navigate the images generated by the artificial dataset.

In [ ]:
explorer = anni.explore()
explorer

## Create

Annotate and label every object in the image. Ipyannotator generates the objects created using indexed labels that starts from 0.

All data is stored as json in the following format:

```json
{
    '../path/to/image1': {
        'bbox': [
            {'x': 1, 'y:' 1, 'width': 1, 'height': 1, 'id': '0'},
            {'x': 2, 'y:' 2, 'width': 2, 'height': 2, 'id': '1'},
        ], 
        'labels': [['Label A'], ['Label B']],
    },
    '../path/to/image2': {
        'bbox': [
            {'x': 1, 'y:' 1, 'width': 1, 'height': 1, 'id': '0'},
        ], 
        'labels': [['Label B']],
    },
    ...
}
```

Every frame has its annotations mapped by the path of the image. Then every bounding box draw in the annotators has it's `x`, `y`, `width`, `height`, `id` properties (as part of the `bbox`) and a label (mapped as `labels`, but with the same index as the object mapped in the `bbox`).

In [ ]:
! rm -rf data/artificial/create_results

**To imitate human work on the current step, let's randomly annotate all the images automatically:**

In [ ]:
anni.output_item = output_
creator = anni.create()

In [ ]:
#SKIP THIS STEP IF YOU ANNOTATE MANUALLY

HELPER = Tutorial(dataset, settings_.project_path)
HELPER.annotate_video_bboxes(creator)

## Improve

Ipyannotator's video annotation tool allows users to:

- Select objects across the frames and join the trajectories drawn.
- Update labels the labels across the whole annotation.

In the example below we have an occlusion colored as black. The rectangle dissapear after the occlusion and appears again with a new object id. The video annotator allow users to join the trajectories of different objects into a new and single one.

In [ ]:
improver = anni.improve()
improver